In [1]:
import pandas as pd

## I. Custom Functions
* add docstrings / make pretty for all finalized functions that are used

In [25]:
def clean_up(sparql_df):
    
    new_director_values = [1 if val == True else 0 for val in sparql_df.directorGender.str.contains('female|non-binary|transgender')]
    new_writer_values = [1 if val == True else 0 for val in sparql_df.screenwriterGender.str.contains('female|non-binary|transgender')]
    new_exec_values = [1 if val == True else 0 for val in sparql_df.executiveproducerGender.str.contains('female|non-binary|transgender')]
    new_producer_values = [1 if val == True else 0 for val in sparql_df.producerGender.str.contains('female|non-binary|transgender')]
    
    sparql_df['director_fnbt'] = new_director_values
    sparql_df['writer_fnbt'] = new_writer_values
    sparql_df['exec_fnbt'] = new_exec_values
    sparql_df['producer_fnbt'] = new_producer_values
    
    sparql_df['overall_fnbt'] = sparql_df['director_fnbt'] + sparql_df['writer_fnbt'] + sparql_df['exec_fnbt'] + sparql_df['producer_fnbt']
    
    cols_to_drop = ['directorName', 'directorGender', 'screenwriterName', 'screenwriterGender', 'executiveproducerName',\
               'executiveproducerGender', 'producerName', 'producerGender']
    sparql_df.drop(columns=cols_to_drop, axis=1, inplace=True)
    sparql_df.sort_values('overall_fnbt', ascending=False, inplace=True)
    sparql_df.drop_duplicates('movieTitle', inplace=True)
    sparql_df.sort_index(inplace=True)
    return sparql_df

#### Probably do not need these functions because `writer` will be dropped from future queries - only need `screenwriter`

In [3]:
# check for all unique values in gender columns, will use my column names within function since they're all the same

def check_unique_gender_values(df):
    gender_columns = ['directorGender', 'writerGender', 'screenwriterGender', 'executiveproducerGender', 'producerGender']
    for col in gender_columns:
        print(col)
        print(df[col].unique())

In [30]:
def locate_rows_in_df_by_values(df, column_to_search, value_list_to_search):
    for val in value_list_to_search:
        print(df.loc[df[column_to_search] == val])

In [14]:
def locate_rows_by_imdb_ids(dataset, name_of_imdb_id_col, imdb_id_list):
    selected_rows = []
    for imdb_id in imdb_id_list:
        row = dataset.loc[dataset[name_of_imdb_id_col] == imdb_id]
        selected_rows.append(row)
    return selected_rows

In [4]:
check_unique_gender_values(movies_2011_and_beyond)

directorGender
['male' 'female' nan 'transgender female' 'non-binary' 'transgender male']
writerGender
[nan 'male']
screenwriterGender
['male' 'female' nan 'transgender female' 'non-binary']
executiveproducerGender
['male' nan 'female']
producerGender
['male' 'female' 'transgender female' nan]


In [6]:
# investigate writerGender because this should probably be dropped and changed to screenwriter
movies_2011_and_beyond.writerName.unique()

array([nan, 'Slavoj Žižek', 'Rana Abrar'], dtype=object)

In [7]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.writerName == 'Rana Abrar']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
15766,Black Briefcase: The Nuclear Trigger,Rana Abrar,male,Rana Abrar,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt7903992
16053,Son Of Kashmir Burhan,Rana Abrar,male,Rana Abrar,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt7675680
16390,The Evil Marriage,Rana Abrar,male,Rana Abrar,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt8614336


In [ ]:
# check to see if these movies are in Bechdel dataset - if not, all 3 will be dropped because there is not much useful
# information for these movies

In [8]:
bechdel_df = pd.read_csv('my_data/bechdel_test_movies.csv')
bechdel_df.head()

,year,title,score,passing,imdb_id,imdb_link
0,2019,Alita: Battle Angel,3,1,tt0437086,http://us.imdb.com/title/tt0437086/
1,2019,American Woman,3,1,tt9109492,http://us.imdb.com/title/tt9109492/
2,2019,Close,3,1,tt5316540,http://us.imdb.com/title/tt5316540/
3,2019,Cold Pursuit,1,0,tt5719748,http://us.imdb.com/title/tt5719748/
4,2019,How to Train Your Dragon: The Hidden World,2,0,tt2386490,http://us.imdb.com/title/tt2386490/


In [16]:
# could turn this imdb_id finder into a function
rana_abrar_imdb_ids = []
for movie in movies_2011_and_beyond.loc[movies_2011_and_beyond.writerName == 'Rana Abrar'].values:
    rana_abrar_imdb_ids.append(movie[-1])
rana_abrar_imdb_ids

['tt7903992', 'tt7675680', 'tt8614336']

In [19]:
locate_rows_by_imdb_ids(bechdel_df, name_of_imdb_id_col='imdb_id', imdb_id_list=rana_abrar_imdb_ids)

[Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: []]

In [21]:
# confirm this function is actually saying there are no matching movies in Bechdel df rather than just being a 
# broken function

bechdel_df.loc[bechdel_df.imdb_id == 'tt7903992']

,year,title,score,passing,imdb_id,imdb_link


In [22]:
bechdel_df.loc[bechdel_df.title == 'The Evil Marriage']

,year,title,score,passing,imdb_id,imdb_link


In [23]:
# check to see if function works on movies that are definitely in both lists

movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Alita: Battle Angel']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
8292,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,James Cameron,male,NaN,NaN,James Cameron,male,170000000.0,359700000.0,tt0437086
8299,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,Laeta Kalogridis,female,NaN,NaN,James Cameron,male,170000000.0,359700000.0,tt0437086
8307,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,James Cameron,male,NaN,NaN,Robert Rodriguez,male,170000000.0,359700000.0,tt0437086
8314,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,Laeta Kalogridis,female,NaN,NaN,Robert Rodriguez,male,170000000.0,359700000.0,tt0437086
8331,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,James Cameron,male,NaN,NaN,Jon Landau,male,170000000.0,359700000.0,tt0437086
8333,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,Laeta Kalogridis,female,NaN,NaN,Jon Landau,male,170000000.0,359700000.0,tt0437086


In [24]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Close']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
15209,Close,Vicky Jewson,female,NaN,NaN,Vicky Jewson,female,NaN,NaN,NaN,NaN,NaN,NaN,tt5316540


In [32]:
movies_to_search = ['Close', 'Alita: Battle Angel', 'American Woman']
locate_rows_in_df_by_values(movies_2011_and_beyond, 'movieTitle', movies_to_search)

      movieTitle  directorName directorGender writerName writerGender  \
15209      Close  Vicky Jewson         female        NaN          NaN   

      screenwriterName screenwriterGender executiveproducerName  \
15209     Vicky Jewson             female                   NaN   

      executiveproducerGender producerName producerGender  budgetAmount  \
15209                     NaN          NaN            NaN           NaN   

       boxofficeAmount     imdbId  
15209              NaN  tt5316540  
               movieTitle      directorName directorGender writerName  \
8292  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8299  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8307  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8314  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8331  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8333  Alita: Battle Angel  Robert Rodriguez           mal

In [34]:
test_imdb_ids = ['tt0437086', 'tt5316540']
locate_rows_by_imdb_ids(movies_2011_and_beyond, 'imdbId', test_imdb_ids)

[               movieTitle      directorName directorGender writerName  \
 8292  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8299  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8307  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8314  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8331  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8333  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 
      writerGender  screenwriterName screenwriterGender executiveproducerName  \
 8292          NaN     James Cameron               male                   NaN   
 8299          NaN  Laeta Kalogridis             female                   NaN   
 8307          NaN     James Cameron               male                   NaN   
 8314          NaN  Laeta Kalogridis             female                   NaN   
 8331          NaN     James Cameron               male                   N

In [35]:
locate_rows_by_imdb_ids(bechdel_df, 'imdb_id', test_imdb_ids)

[   year                title  score  passing    imdb_id  \
 0  2019  Alita: Battle Angel      3        1  tt0437086   
 
                              imdb_link  
 0  http://us.imdb.com/title/tt0437086/  ,
    year  title  score  passing    imdb_id                            imdb_link
 2  2019  Close      3        1  tt5316540  http://us.imdb.com/title/tt5316540/]

In [33]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'American Woman']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId


In [36]:
# well the outputs don't look pretty, but at least they work!

In [37]:
# the whole point of all that was just to see if I could drop the writerName and writerGender columns

In [38]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.writerName == 'Slavoj Žižek']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
11007,The Pervert's Guide to Ideology,Sophie Fiennes,female,Slavoj Žižek,male,Slavoj Žižek,male,NaN,NaN,NaN,NaN,NaN,NaN,tt2152198


In [40]:
# check to see if this movie is in the bechdel dataset
locate_rows_by_imdb_ids(bechdel_df, 'imdb_id', 'tt2152198')

[Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: []]

In [41]:
bechdel_df.loc[bechdel_df.title == 'The Pervert\'s Guide to Ideology']

,year,title,score,passing,imdb_id,imdb_link


In [42]:
# drop writerName and writerGender from movies_200 df
movies_2011_and_beyond.drop(columns=['writerName', 'writerGender'], axis=1, inplace=True)

In [43]:
movies_2011_and_beyond.head()

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
1,Albert Nobbs,Rodrigo García,male,Glenn Close,female,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
2,Albert Nobbs,Rodrigo García,male,George Moore,male,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
3,300: Rise of an Empire,Noam Murro,male,Kurt Johnstad,male,NaN,NaN,Thomas Tull,male,110000000.0,337580051.0,tt1253863
4,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343


#### Convert gender columns to a boolean `FTNB` column to include cis and transgender females, transgender males, and non-binary people
* Need to be able to drop duplicate rows so that there is only one row for each movie
* Each movie row would indicate whether one of its directors, writers, or producers is of an underrepresented gender
* Would like an additional column for presence of underrepresented gender in ANY of those listed positions

In [117]:
# new boolean gender columns
# drop name and other gender columns
# add overall gender column
# sort by descending overall values
# drop duplicates, keeping first
# will have to check writer columns in one other dataset, then might drop that from the other queries to download

In [133]:
bechdel_df.loc[bechdel_df.title == 'Star Wars: The Last Jedi']

,year,title,score,passing,imdb_id,imdb_link
349,2017,Star Wars: The Last Jedi,3,1,tt2527336,http://us.imdb.com/title/tt2527336/


In [134]:
# The Last Jedi should be passing once all rows are considered

In [163]:
# please work please work please work
# clean_up(testing)

In [159]:
testing.loc[testing.movieTitle == 'Star Wars: The Last Jedi']

,movieTitle,writerName,writerGender,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
4585,Star Wars: The Last Jedi,NaN,NaN,200000000.0,42577974.0,tt2527336,0,0,0,1,1


In [ ]:
# yay!

## II. Create new movie dataset from all downloaded queries

### A. Movies released in 2011 - present

In [164]:
cleaned_2011_and_beyond = pd.read_csv('my_data/movies_2011_and_beyond.csv')
cleaned_2011_and_beyond.head()

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Fast Five,Neal H. Moritz,male,NaN,NaN,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
1,Albert Nobbs,Rodrigo García,male,NaN,NaN,Glenn Close,female,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
2,Albert Nobbs,Rodrigo García,male,NaN,NaN,George Moore,male,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
3,300: Rise of an Empire,Noam Murro,male,NaN,NaN,Kurt Johnstad,male,NaN,NaN,Thomas Tull,male,110000000.0,337580051.0,tt1253863
4,Fast Five,Neal H. Moritz,male,NaN,NaN,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343


In [166]:
cleaned_2011_and_beyond.drop(columns=['writerName', 'writerGender'], axis=1, inplace=True)

In [167]:
clean_up(cleaned_2011_and_beyond)

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
0,Fast Five,NaN,626100000.0,tt1596343,0,0,0,0,0
1,Albert Nobbs,NaN,NaN,tt1602098,0,1,0,1,2
8,Lore,NaN,NaN,tt1996310,1,1,0,1,3
11,Get the Gringo,NaN,NaN,tt1567609,0,0,0,0,0
40,Airborne,NaN,NaN,tt1827354,0,0,0,0,0
43,300: Rise of an Empire,110000000.0,337580051.0,tt1253863,0,0,0,1,1
53,Super 8,NaN,260100000.0,tt1650062,0,0,0,0,0
54,Puncture,NaN,NaN,tt1582248,0,0,0,0,0
55,Chinese Zodiac,NaN,NaN,tt1424310,0,0,0,0,0
58,The Double,NaN,NaN,tt1646980,0,0,0,0,0


### B. Movies from 2005 to 2010

In [168]:
cleaned_movies_2005_to_2010 = pd.read_csv('my_data/movies_2005_2010_inclusive.csv')
cleaned_movies_2005_to_2010.head()

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,The Reader,Stephen Daldry,male,NaN,NaN,Bernhard Schlink,male,Bob Weinstein,male,Anthony Minghella,male,NaN,NaN,tt0976051
1,King Kong,Peter Jackson,male,NaN,NaN,Peter Jackson,male,Robert Zemeckis,male,Peter Jackson,male,207000000.0,550517357.0,tt0360717
2,The Reader,Stephen Daldry,male,NaN,NaN,Bernhard Schlink,male,Harvey Weinstein,male,Donna Gigliotti,female,NaN,NaN,tt0976051
3,King Kong,Peter Jackson,male,NaN,NaN,Fran Walsh,female,Robert Zemeckis,male,Fran Walsh,female,207000000.0,550517357.0,tt0360717
4,King Kong,Peter Jackson,male,NaN,NaN,Edgar Wallace,male,Robert Zemeckis,male,Fran Walsh,female,207000000.0,550517357.0,tt0360717


#### Drop `writer` columns and `clean_up` df

In [171]:
cleaned_movies_2005_to_2010.drop(columns=['writerName', 'writerGender'], axis=1, inplace=True)

In [172]:
clean_up(cleaned_movies_2005_to_2010)

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
14,The King's Speech,NaN,414200000.0,tt1504320,0,0,0,0,0
22,King Kong,207000000.0,550517357.0,tt0360717,0,1,0,1,2
31,Twilight,NaN,393600000.0,tt1099212,1,1,0,0,2
33,The Reader,NaN,NaN,tt0976051,0,0,0,1,1
40,Valkyrie,80000000.0,200000000.0,tt0985699,0,0,0,0,0
42,Must Love Dogs,NaN,NaN,tt0417001,0,0,0,0,0
44,Crank,NaN,NaN,tt0479884,0,0,0,0,0
46,Feast of Love,NaN,NaN,tt0800027,0,0,0,0,0
47,300,65000000.0,456068181.0,tt0416449,0,0,0,0,0
49,Obsessed,NaN,NaN,tt1198138,0,0,0,0,0


### C. Merge first two datasets

In [173]:
movies_2005_and_beyond = pd.concat([cleaned_2011_and_beyond, cleaned_movies_2005_to_2010], ignore_index=True)

In [174]:
movies_2005_and_beyond.head()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
0,Fast Five,NaN,626100000.0,tt1596343,0,0,0,0,0
1,Albert Nobbs,NaN,NaN,tt1602098,0,1,0,1,2
2,Lore,NaN,NaN,tt1996310,1,1,0,1,3
3,Get the Gringo,NaN,NaN,tt1567609,0,0,0,0,0
4,Airborne,NaN,NaN,tt1827354,0,0,0,0,0


In [175]:
movies_2005_and_beyond.tail()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
16785,Must Read After My Death,NaN,NaN,tt1249414,0,0,0,0,0
16786,The Upside of Anger,NaN,NaN,tt0365885,0,0,0,0,0
16787,10th & Wolf,NaN,NaN,tt0360323,0,0,0,1,1
16788,A Simple Noodle Story,NaN,NaN,tt1428556,0,0,0,0,0
16789,My Apocalypse,NaN,NaN,tt0119970,0,0,0,0,0


In [176]:
movies_2005_and_beyond.shape

(16790, 9)

### D. Clean and append movies from 1999 to 2004

In [177]:
cleaned_movies_1999_to_2004 = pd.read_csv('my_data/movies_1999_2004_inclusive.csv')
cleaned_movies_1999_to_2004.head()

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,The Hunted,William Friedkin,male,NaN,NaN,Art Monterastelli,male,NaN,NaN,James Jacks,male,NaN,NaN,tt0269347
1,Gangs of New York,Martin Scorsese,male,NaN,NaN,Jay Cocks,male,NaN,NaN,Harvey Weinstein,male,97000000.0,193772504.0,tt0217505
2,Resident Evil,Paul W. S. Anderson,male,NaN,NaN,Paul W. S. Anderson,male,NaN,NaN,Samuel Hadida,male,33000000.0,102441078.0,tt0120804
3,My Date with Drew,Brian Herzlinger,male,NaN,NaN,NaN,NaN,NaN,NaN,Brian Herzlinger,male,NaN,NaN,tt0378407
4,Runaway Jury,Gary Fleder,male,NaN,NaN,Brian Koppelman,male,NaN,NaN,Arnon Milchan,male,NaN,NaN,tt0313542


In [178]:
cleaned_movies_1999_to_2004.drop(columns=['writerName', 'writerGender'], axis=1, inplace=True)

In [179]:
clean_up(cleaned_movies_1999_to_2004)
cleaned_movies_1999_to_2004.tail()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
8029,Venus and Mars,NaN,NaN,tt0205498,0,0,0,0,0
8031,101 Reykjavík,NaN,1.264040e+02,tt0237993,0,0,0,0,0
8035,Gosford Park,19800000.0,8.770000e+07,tt0280707,0,0,0,0,0
8036,Bowling for Columbine,NaN,NaN,tt0310793,0,0,0,0,0
8040,Joy Ride,NaN,NaN,tt0206314,0,0,0,0,0


In [180]:
cleaned_movies_1999_to_2004[500:550]

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
1902,Undisputed,NaN,NaN,tt0281322,0,0,0,0,0
1903,The In-Laws,NaN,NaN,tt0314786,0,0,0,0,0
1904,True Crime,NaN,NaN,tt0139668,0,0,0,0,0
1905,Edges of the Lord,NaN,NaN,tt0245090,0,0,0,0,0
1907,The Statement,NaN,NaN,tt0340376,0,0,0,0,0
1910,Jack Frost,NaN,NaN,tt0141109,0,0,0,0,0
1911,A Midsummer Night's Dream,NaN,NaN,tt0140379,0,1,0,0,1
1918,Iris,NaN,NaN,tt0280778,0,0,0,0,0
1920,Ghost Ship,NaN,NaN,tt0288477,0,0,0,0,0
1922,Ararat,NaN,NaN,tt0273435,0,0,0,0,0


In [181]:
movies_1999_and_beyond = pd.concat([movies_2005_and_beyond, cleaned_movies_1999_to_2004], ignore_index=True)

In [182]:
movies_1999_and_beyond.shape

(21499, 9)

### E. Clean and append movies from 1980 to 1998

In [183]:
movies_1980_to_1998 = pd.read_csv('my_data/movies_1980_to_1998_inclusive.csv')
movies_1980_to_1998.head()

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Beloved,Jonathan Demme,male,Richard LaGravenese,male,Ron Bozman,male,Edward Saxon,male,NaN,NaN,tt0120603
1,First Blood,Ted Kotcheff,male,William Sackheim,male,NaN,NaN,Mario Kassar,male,15000000.0,125212904.0,tt0083944
2,Beloved,Jonathan Demme,male,Toni Morrison,female,Ron Bozman,male,Edward Saxon,male,NaN,NaN,tt0120603
3,Evita,Alan Parker,male,Oliver Stone,male,NaN,NaN,Alan Parker,male,NaN,NaN,tt0116250
4,Beloved,Jonathan Demme,male,Adam Brooks,male,Ron Bozman,male,Jonathan Demme,male,NaN,NaN,tt0120603


In [184]:
clean_up(movies_1980_to_1998)
movies_1980_to_1998.tail()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
14397,Top Secret!,NaN,NaN,tt0088286,0,0,0,0,0
14398,Denise Calls Up,NaN,NaN,tt0112844,0,0,0,0,0
14399,Serial Mom,NaN,NaN,tt0111127,0,0,0,0,0
14400,Swoon,NaN,NaN,tt0105508,0,0,0,1,1
14401,Theodore Rex,NaN,NaN,tt0114658,0,0,0,0,0


In [185]:
movies_1980_and_beyond = pd.concat([movies_1999_and_beyond, movies_1980_to_1998], ignore_index=True)
movies_1980_and_beyond.shape

(30735, 9)

### E. Movies from 1950 to 1979

In [186]:
movies_1950_to_1979 = pd.read_csv('my_data/movies_1950_to_1979_inclusive.csv')
movies_1950_to_1979.head()

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,To Hell and Back,Henry Mancini,male,Audie Murphy,male,NaN,NaN,Aaron Rosenberg,male,NaN,NaN,tt0048729
1,To Hell and Back,Irving Gertz,male,Audie Murphy,male,NaN,NaN,Henry Mancini,male,NaN,NaN,tt0048729
2,Two English Girls,François Truffaut,male,François Truffaut,male,NaN,NaN,Marcel Berbert,male,NaN,NaN,tt0066989
3,To Hell and Back,Maury Gertsman,male,Audie Murphy,male,NaN,NaN,Irving Gertz,male,NaN,NaN,tt0048729
4,To Hell and Back,Walter Bedell Smith,male,Audie Murphy,male,NaN,NaN,Irving Gertz,male,NaN,NaN,tt0048729


In [187]:
clean_up(movies_1950_to_1979)
movies_1950_to_1979.tail()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
15162,"I, Maureen",NaN,NaN,tt0180751,1,0,0,0,1
15163,Oliver Twist,NaN,NaN,tt0272226,0,0,0,0,0
15164,Man on a Bus,NaN,NaN,tt0183487,0,0,0,0,0
15165,"A Man, a Woman, and a Killer",NaN,NaN,tt0073348,0,0,0,0,0
15166,None but the Brave,NaN,NaN,tt0053831,0,0,0,0,0


In [188]:
movies_1950_and_beyond = pd.concat([movies_1980_and_beyond, movies_1950_to_1979], ignore_index=True)
movies_1950_and_beyond.shape

(41493, 9)

### F. Movies 1949 and Earlier

In [189]:
movies_1949_and_earlier = pd.read_csv('my_data/movies_1949_and_earlier.csv')
movies_1949_and_earlier.head()

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,British Agent,Michael Curtiz,male,NaN,NaN,NaN,NaN,Michael Curtiz,male,NaN,NaN,tt0024915
1,The Third Man,Carol Reed,male,Alexander Korda,male,NaN,NaN,David O. Selznick,male,NaN,NaN,tt0041959
2,Dracula,Karl Freund,male,Garrett Fort,male,NaN,NaN,Tod Browning,male,NaN,NaN,tt0021814
3,All Quiet on the Western Front,Lewis Milestone,male,C. Gardner Sullivan,male,NaN,NaN,Carl Laemmle,male,1200000.0,NaN,tt0020629
4,Go Into Your Dance,Archie Mayo,male,Earl Baldwin,male,NaN,NaN,Jack Warner,male,NaN,NaN,tt0026418


In [190]:
clean_up(movies_1949_and_earlier)
movies_1949_and_earlier.tail()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
18753,Foolish Wives,NaN,NaN,tt0013140,0,0,0,0,0
18755,Cairo,NaN,NaN,tt0034565,0,0,0,0,0
18757,The Vanishing Legion,NaN,NaN,tt0022531,0,0,0,0,0
18758,The Citadel,NaN,NaN,tt0029995,0,1,0,0,1
18760,Things to Come,NaN,NaN,tt0028358,0,0,0,0,0


In [191]:
# just curious
movies_1949_and_earlier.overall_fnbt.unique()

array([0, 1, 2, 3])

In [193]:
movies_1949_and_earlier.overall_fnbt.value_counts().nlargest(5)

0    11756
1     1243
2       57
3       15
Name: overall_fnbt, dtype: int64

In [194]:
all_the_movies = pd.concat([movies_1950_and_beyond, movies_1949_and_earlier], ignore_index=True)

In [195]:
all_the_movies.shape

(54564, 9)

In [196]:
all_the_movies.to_csv('my_data/movies_cleaned_with_new_columns.csv')

### G. New Query - all Movies on Wikidata with a Bechdel ID
Still having trouble finding a good percentage of the movies in the Bechdel dataset, so tried a new query

In [30]:
new_bechdel_query = pd.read_csv('my_data/bechdel_id_query.csv')
new_bechdel_query.head()

,title,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Ute Emmerich,female,Roland Emmerich,male,NaN,817400891.0,tt0116629
1,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Ute Emmerich,female,Dean Devlin,male,NaN,817400891.0,tt0116629
2,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Roland Emmerich,male,Roland Emmerich,male,NaN,817400891.0,tt0116629
3,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Roland Emmerich,male,Dean Devlin,male,NaN,817400891.0,tt0116629
4,Independence Day,Roland Emmerich,male,Dean Devlin,male,Ute Emmerich,female,Roland Emmerich,male,NaN,817400891.0,tt0116629


In [31]:
new_bechdel_query.title.nunique()

7161

#### Apply cleaning methods used on previous datasets

In [32]:
# change title column to movieTitle so the function will work
new_bechdel_query.rename(index=str, columns={"title": "movieTitle"}, inplace=True)

In [33]:
new_bechdel_query.head()

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Ute Emmerich,female,Roland Emmerich,male,NaN,817400891.0,tt0116629
1,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Ute Emmerich,female,Dean Devlin,male,NaN,817400891.0,tt0116629
2,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Roland Emmerich,male,Roland Emmerich,male,NaN,817400891.0,tt0116629
3,Independence Day,Roland Emmerich,male,Roland Emmerich,male,Roland Emmerich,male,Dean Devlin,male,NaN,817400891.0,tt0116629
4,Independence Day,Roland Emmerich,male,Dean Devlin,male,Ute Emmerich,female,Roland Emmerich,male,NaN,817400891.0,tt0116629


In [34]:
clean_up(new_bechdel_query)

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
10004,The Dark Crystal,NaN,NaN,tt0083791,0,0,0,0,0
10005,The Thin Man Goes Home,NaN,NaN,tt0037365,0,0,0,0,0
10007,Picture Perfect,NaN,NaN,tt0119896,0,1,0,0,1
10009,The Cruel Sea,NaN,NaN,tt0045659,0,0,0,0,0
10010,Our Hospitality,NaN,NaN,tt0014341,0,0,0,0,0
10015,Little Caesar,NaN,NaN,tt0021079,0,0,0,0,0
10016,The Long Walk Home,NaN,NaN,tt0100046,0,0,0,0,0
10017,Cléo from 5 to 7,NaN,NaN,tt0055852,1,1,0,0,2
10018,The Spanish Gardener,NaN,NaN,tt0049787,0,0,0,0,0
10021,Antitrust,NaN,NaN,tt0218817,0,0,0,0,0


## III. Add to Bechdel Test dataset

In [2]:
bechdel_df = pd.read_csv('my_data/bechdel_test_movies.csv')
bechdel_df.head()

,year,title,score,passing,imdb_id,imdb_link
0,2019,Alita: Battle Angel,3,1,tt0437086,http://us.imdb.com/title/tt0437086/
1,2019,American Woman,3,1,tt9109492,http://us.imdb.com/title/tt9109492/
2,2019,Close,3,1,tt5316540,http://us.imdb.com/title/tt5316540/
3,2019,Cold Pursuit,1,0,tt5719748,http://us.imdb.com/title/tt5719748/
4,2019,How to Train Your Dragon: The Hidden World,2,0,tt2386490,http://us.imdb.com/title/tt2386490/


In [3]:
bechdel_df.drop('imdb_link', axis=1, inplace=True)

In [4]:
bechdel_df.tail()

,year,title,score,passing,imdb_id
8045,1896,Une nuit terrible,0,0,tt0000131
8046,1895,"The Execution of Mary, Queen of Scots",0,0,tt0132134
8047,1895,Tables Turned on the Gardener,0,0,tt0000014
8048,1892,Pauvre Pierrot,0,0,tt0000003
8049,1888,Roundhay Garden Scene,0,0,tt0392728


In [199]:
for imdb in bechdel_df.imdb_id.values:
    print(imdb)
    break

tt0437086


In [5]:
all_the_movies = pd.read_csv('my_data/movies_cleaned_with_new_columns.csv')
all_the_movies.head()

,Unnamed: 0,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
0,0,Fast Five,NaN,626100000.0,tt1596343,0,0,0,0,0
1,1,Albert Nobbs,NaN,NaN,tt1602098,0,1,0,1,2
2,2,Lore,NaN,NaN,tt1996310,1,1,0,1,3
3,3,Get the Gringo,NaN,NaN,tt1567609,0,0,0,0,0
4,4,Airborne,NaN,NaN,tt1827354,0,0,0,0,0


In [6]:
all_the_movies.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
all_the_movies.tail()

,movieTitle,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt,overall_fnbt
54559,Foolish Wives,NaN,NaN,tt0013140,0,0,0,0,0
54560,Cairo,NaN,NaN,tt0034565,0,0,0,0,0
54561,The Vanishing Legion,NaN,NaN,tt0022531,0,0,0,0,0
54562,The Citadel,NaN,NaN,tt0029995,0,1,0,0,1
54563,Things to Come,NaN,NaN,tt0028358,0,0,0,0,0


In [8]:
bechdel_ids = bechdel_df.imdb_id.values
the_other_ids = all_the_movies.imdbId.values

In [9]:
shared_ids = set(bechdel_ids) & set(the_other_ids)
len(shared_ids)

5471

In [10]:
# check to see what might be missing 

In [11]:
type(shared_ids)

set

In [14]:
missing_movies = []
for bechdel_id in bechdel_ids:
    if bechdel_id not in list(shared_ids):
        missing_movies.append(bechdel_id)

In [15]:
len(missing_movies)

2577

In [16]:
2577 + 5471

8048

In [17]:
bechdel_df.shape

(8050, 5)

In [18]:
missing_movies[:10]

['tt9109492',
 'tt2386490',
 'tt3513498',
 'tt6078866',
 'tt7605074',
 'tt9358120',
 'tt6142496',
 'tt1620680',
 'tt4244998',
 'tt8020896']

In [21]:
for missing_id in missing_movies[:10]:
    print(bechdel_df.loc[bechdel_df.imdb_id == missing_id])

   year           title  score  passing    imdb_id
1  2019  American Woman      3        1  tt9109492
   year                                       title  score  passing    imdb_id
4  2019  How to Train Your Dragon: The Hidden World      2        0  tt2386490
   year                              title  score  passing    imdb_id
6  2019  The Lego Movie 2: The Second Part      3        1  tt3513498
   year title  score  passing    imdb_id
9  2019  Soni      3        1  tt6078866
    year                                  title  score  passing    imdb_id
10  2019  The Wandering Earth (Liu Lang Di Qiu)      3        1  tt7605074
    year                  title  score  passing    imdb_id
11  2019  We Are Little Zombies      2        0  tt9358120
    year       title  score  passing    imdb_id
12  2018  6 Balloons      3        1  tt6142496
    year              title  score  passing    imdb_id
17  2018  A Wrinkle in Time      3        1  tt1620680
    year  title  score  passing    imdb_id
1

In [35]:
new_query_ids = new_bechdel_query.imdbId.values

In [36]:
shared_ids = set(bechdel_ids) & set(new_query_ids)
len(shared_ids)

7147

In [ ]:
# not perfect - but so much better 